In [1]:
import shutil
import subprocess
import glob
from tqdm import tqdm
import numpy as np
import os
import argparse


In [2]:
import sys
sys.executable

'/home/chongke/anaconda3/envs/my_env/bin/python'

In [3]:

import torch
from torch import nn
import torch.nn.functional as F
import pretrainedmodels
from pretrainedmodels import utils

C, H, W = 3, 224, 224

In [4]:
args = {'gpu': '0', 'output_dir': 'data/feats/demo', 'n_frame_steps': 40, 'video_path':'data/train-video', 'model': 'resnet152'}

In [5]:
args

{'gpu': '0',
 'output_dir': 'data/feats/demo',
 'n_frame_steps': 40,
 'video_path': 'data/train-video',
 'model': 'resnet152'}

In [6]:
os.environ['CUDA_VISIBLE_DEVICES'] = args['gpu']

In [7]:
params = args

In [8]:
C, H, W = 3, 224, 224
model = pretrainedmodels.resnet152(pretrained='imagenet')
load_image_fn = utils.LoadTransformImage(model)

In [9]:
model.last_linear = utils.Identity()
model = nn.DataParallel(model)
model = model.cuda()

In [10]:
model.eval();

In [11]:
dir_fc = params['output_dir']

In [12]:
if not os.path.isdir(dir_fc):
    os.mkdir(dir_fc)
print("save video feats to %s" % (dir_fc))

save video feats to data/feats/demo


In [13]:
video_list = glob.glob(os.path.join(params['video_path'], '*.mp4'))

In [14]:
video = 'data/sample-video/video1211.mp4'

In [15]:
video_id = video.split("/")[-1].split(".")[0]

In [16]:
dst = params['model'] + '_' + video_id

In [17]:
dst

'resnet152_video1211'

In [18]:
with open(os.devnull, "w") as ffmpeg_log:
    if os.path.exists(dst):
        print(" cleanup: " + dst + "/")
        shutil.rmtree(dst)
    os.makedirs(dst)
    video_to_frames_command = ["ffmpeg",
                               # (optional) overwrite output file if it exists
                               '-y',
                               '-i', video,  # input file
                               '-vf', "scale=400:300",  # input file
                               '-qscale:v', "2",  # quality for JPEG
                               '{0}/%06d.jpg'.format(dst)]
    subprocess.call(video_to_frames_command,
                    stdout=ffmpeg_log, stderr=ffmpeg_log)

 cleanup: resnet152_video1211/


In [19]:
image_list = sorted(glob.glob(os.path.join(dst, '*.jpg')))
samples = np.round(np.linspace(
            0, len(image_list) - 1, params['n_frame_steps']))
image_list = [image_list[int(sample)] for sample in samples]

In [20]:
images = torch.zeros((len(image_list), C, H, W))
for iImg in range(len(image_list)):
    img = load_image_fn(image_list[iImg])
    images[iImg] = img

In [21]:
with torch.no_grad():
    fc_feats = model(images.cuda()).squeeze()

In [22]:
 img_feats = fc_feats.cpu().numpy()
# Save the inception features
outfile = os.path.join(dir_fc, video_id + '.npy')
np.save(outfile, img_feats)
# cleanup
shutil.rmtree(dst)